<a href="https://colab.research.google.com/github/Amitukra/Data-Analysis-Process/blob/main/Data%20extraction/Task2_Growify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import re

In [ ]:
df = pd.read_csv("/content/Google Sheet Growify.csv")

In [ ]:
df.head()

,Business Name,Email,Phone Number,Address,Website
0,SEO Ashburn,info@SEOAshburn.com,(254) 753-7395,"44050 Ashburn Shopping Plz Suite 195-636, Ashb...",seoashburn.com
1,Adzeem Digital Marketing,info@adzeem.com,(410) 995-7776,"8000 Towers Crescent Dr 13th Floor, Vienna, VA...",adzeem.com


In [ ]:
# Exctract About Content...

def get_about_us_content(website):
    base_url = f"https://{website.strip('/')}/"
    paths = ["about", "about-us", "company/about", "who-we-are"]

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
                      " (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
    }

    for path in paths:
        try:
            full_url = urljoin(base_url, path)
            response = requests.get(full_url, headers=headers, timeout=10)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                paragraphs = soup.find_all(['p', 'div'], text=True)
                text = ' '.join(p.get_text(strip=True) for p in paragraphs)
                clean_text = re.sub(r'\s+', ' ', text).strip()
                if len(clean_text) > 100:
                    return clean_text
        except:
            continue
    return "Not found"

df["About Content"] = df["Website"].apply(get_about_us_content)

<ipython-input-13-37930473f2d5>:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  paragraphs = soup.find_all(['p', 'div'], text=True)


In [ ]:
df

,Business Name,Email,Phone Number,Address,Website,About Content
0,SEO Ashburn,info@SEOAshburn.com,(254) 753-7395,"44050 Ashburn Shopping Plz Suite 195-636, Ashb...",seoashburn.com,We have a diverse team ready to take on your u...
1,Adzeem Digital Marketing,info@adzeem.com,(410) 995-7776,"8000 Towers Crescent Dr 13th Floor, Vienna, VA...",adzeem.com,Adzeem helps their clients show their awesomen...


In [ ]:
# Extract Keywords using (TF-IDF)......

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def extract_keywords(text, top_n=10):
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform([text])
    scores = zip(vectorizer.get_feature_names_out(), tfidf_matrix.toarray()[0])
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    keywords = [word for word, score in sorted_scores[:top_n]]
    return keywords

df["Keywords"] = df["About Content"].apply(lambda x: extract_keywords(x) if x != "Not found" else [])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df

,Business Name,Email,Phone Number,Address,Website,About Content,Keywords
0,SEO Ashburn,info@SEOAshburn.com,(254) 753-7395,"44050 Ashburn Shopping Plz Suite 195-636, Ashb...",seoashburn.com,We have a diverse team ready to take on your u...,"[marketing, manager, online, business, maximiz..."
1,Adzeem Digital Marketing,info@adzeem.com,(410) 995-7776,"8000 Towers Crescent Dr 13th Floor, Vienna, VA...",adzeem.com,Adzeem helps their clients show their awesomen...,"[digital, adzeem, marketing, services, agency,..."


In [ ]:
# Generate Content from Keywords (Basic Version).....

def generate_summary(keywords):
    if not keywords:
        return "No content available."
    return f"Our brand focuses on {', '.join(keywords[:5])}, delivering quality and innovation in every step."

df["Generated Content"] = df["Keywords"].apply(generate_summary)

# RESULT
print(df[["Business Name", "Website", "Keywords", "Generated Content"]])

              Business Name         Website  \
0               SEO Ashburn  seoashburn.com   
1  Adzeem Digital Marketing      adzeem.com   

                                            Keywords  \
0  [marketing, manager, online, business, maximiz...   
1  [digital, adzeem, marketing, services, agency,...   

                                   Generated Content  
0  Our brand focuses on marketing, manager, onlin...  
1  Our brand focuses on digital, adzeem, marketin...  


In [ ]:
df

,Business Name,Email,Phone Number,Address,Website,About Content,Keywords,Generated Content
0,SEO Ashburn,info@SEOAshburn.com,(254) 753-7395,"44050 Ashburn Shopping Plz Suite 195-636, Ashb...",seoashburn.com,We have a diverse team ready to take on your u...,"[marketing, manager, online, business, maximiz...","Our brand focuses on marketing, manager, onlin..."
1,Adzeem Digital Marketing,info@adzeem.com,(410) 995-7776,"8000 Towers Crescent Dr 13th Floor, Vienna, VA...",adzeem.com,Adzeem helps their clients show their awesomen...,"[digital, adzeem, marketing, services, agency,...","Our brand focuses on digital, adzeem, marketin..."
